In [ ]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from transformers import ViTForImageClassification
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm

# -------------------- Image Processing Functions --------------------

def static2dynamic(image_data):
    """
    Convert a static borehole image to a dynamic one by remapping pixel values
    based on the 3rd and 97th percentiles to maximize contrast.
    """
    q3 = np.nanquantile(image_data[image_data < 255.0], 0.03)
    q97 = np.nanquantile(image_data[image_data < 255.0], 0.97)
    image_data_dynamic = image_data.apply(lambda x: 255 * (x - q3) / (q97 - q3))
    image_data_dynamic[image_data_dynamic < 0] = 0
    image_data_dynamic[image_data_dynamic > 255] = 255
    return image_data_dynamic

def create_multi_track_image(borehole_image, logs_dict, depth_range, log_names):
    """
    Create a composite multi-track image that combines the dynamic borehole image
    with selected well log curves.
    
    Parameters:
        borehole_image: The dynamic borehole image (DataFrame) data.
        logs_dict: A dictionary containing the well log data (e.g., GR, ILD, etc.).
        depth_range: Tuple (top_depth, bottom_depth) specifying the depth window.
        log_names: List of keys (strings) indicating which logs to plot.
    
    Returns:
        img: The composite multi-track image as a NumPy array.
    """
    plt.ioff()  # Turn off interactive plotting
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5), gridspec_kw={'width_ratios': [3, 1]})
    
    # Plot the dynamic borehole image on the left
    ax1.imshow(borehole_image, cmap='gray', aspect='auto', extent=[0, 360, depth_range[1], depth_range[0]])
    ax1.axis('off')
    
    # Plot the well log curves on the right
    depth_idx = np.linspace(depth_range[0], depth_range[1], len(logs_dict[log_names[0]]))
    colors = ['red', 'blue', 'green', 'purple', 'orange']
    for i, name in enumerate(log_names):
        data = logs_dict[name]
        normalized = (data - np.min(data)) / (np.max(data) - np.min(data))
        ax2.plot(normalized, depth_idx, color=colors[i])
    ax2.axis('off')
    
    # Convert the figure to an image array
    fig.canvas.draw()
    w, h = fig.canvas.get_width_height()
    buf = np.frombuffer(fig.canvas.buffer_rgba(), dtype=np.uint8)
    img = buf.reshape(h, w, 4)[:, :, :3]
    plt.close(fig)
    plt.ion()  # Turn interactive plotting back on
    return img

def get_transforms(train=True):
    """
    Returns an Albumentations Compose transform for image resizing and normalization.
    """
    return A.Compose([
        A.Resize(224, 224),
        A.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225]),
        ToTensorV2()
    ])

# -------------------- Model and Ensemble Functions --------------------

class EnsembleModel:
    """
    Ensemble of models. Predictions from each model are averaged.
    """
    def __init__(self, models):
        self.models = models

    def eval(self):
        for model in self.models:
            model.eval()

    def predict(self, x):
        outputs = []
        with torch.no_grad():
            for model in self.models:
                output = model(x).logits
                probs = torch.softmax(output, dim=1)
                outputs.append(probs)
        # Average predictions over the ensemble
        return torch.mean(torch.stack(outputs), dim=0)

def load_ensemble_model(ensemble_pth_path, device):
    """
    Loads the ensemble model checkpoint from the specified .pth file and returns
    an EnsembleModel instance.
    """
    checkpoint = torch.load(ensemble_pth_path, map_location=device)
    model_states = checkpoint['model_states']
    models = []
    for state in model_states:
        model = ViTForImageClassification.from_pretrained(
            'google/vit-base-patch16-224',
            num_labels=2,
            ignore_mismatched_sizes=True
        ).to(device)
        model.load_state_dict(state)
        model.eval()
        models.append(model)
    ensemble = EnsembleModel(models)
    ensemble.eval()
    return ensemble

# -------------------- Test Views Generation --------------------

def generate_test_views(well_images, well_logs, well_list=['T1', 'T2']):
    """
    Generates multi-track test images using a sliding window approach for each test well.
    
    Parameters:
        well_images (dict): Dictionary with keys like 'well_T1', 'well_T2', containing DataFrames.
        well_logs (dict): Dictionary with corresponding well log DataFrames.
        well_list (list): List of test well identifiers (e.g., ['T1', 'T2']).
    
    Returns:
        test_views (dict): Dictionary where each key (e.g., 'T1') maps to a list of tuples
                           (multi-track image, center depth).
    """
    test_views = {}
    for well in well_list:
        well_name = f"well_{well}"
        depths = well_images[well_name].index
        test_views[well] = []
        # Use a sliding window: here window size = 1000 rows, step = 500 rows.
        for i in range(0, len(depths) - 1000, 500):
            center_depth = depths[i] + 1.0
            top_depth = center_depth - 1.0
            bottom_depth = center_depth + 1.0
            image_data = well_images[well_name].iloc[i:i+1000]
            image_data_dynamic = static2dynamic(image_data)
            logs_dict = {
                'GR': well_logs[well_name].GR,
                'ILD': np.log10(well_logs[well_name].ILD),
                'NPOR': well_logs[well_name].NPOR_LIM,
                'RHOB': well_logs[well_name].RHO,
                'DTC': well_logs[well_name].DT
            }
            multi_track = create_multi_track_image(
                image_data_dynamic,
                logs_dict,
                (top_depth, bottom_depth),
                ['GR', 'ILD', 'NPOR', 'RHOB', 'DTC']
            )
            test_views[well].append((multi_track, center_depth))
    return test_views

# -------------------- Inference Function --------------------

def infer_test_wells(ensemble_pth_path, test_views, transform, device, conf_threshold=0.8):
    """
    Loads the ensemble model from the specified checkpoint and performs inference on the
    test wells (T1 and T2) using the generated test views.
    
    Parameters:
        ensemble_pth_path (str): Path to the saved ensemble checkpoint file.
        test_views (dict): Dictionary with keys 'T1' and 'T2' containing lists of tuples 
                           (multi-track image, center depth).
        transform (callable): Transform function to convert images to tensors.
        device (torch.device): Device to run inference on.
        conf_threshold (float): Confidence threshold for fracture detection.
    
    Returns:
        predictions (dict): Dictionary mapping each well ('T1' and 'T2') to a list of tuples
                            (depth, probability) for predicted fractures.
    """
    ensemble = load_ensemble_model(ensemble_pth_path, device)
    predictions = {well: [] for well in ['T1', 'T2']}
    
    for well in ['T1', 'T2']:
        print(f"\nMaking predictions for well {well}...")
        for image, depth in test_views[well]:
            image_tensor = transform(image=image)['image'].unsqueeze(0).to(device)
            with torch.no_grad():
                probs = ensemble.predict(image_tensor)
            # Assuming index 1 corresponds to the fracture class
            prob = probs[0][1].item()
            if prob > conf_threshold:
                predictions[well].append((depth, prob))
    return predictions

# -------------------- Submission CSV Creation --------------------

def create_submission_csv(predictions, output_csv_path, num_predictions_per_well=15):
    """
    Creates a submission CSV file from the predictions dictionary.
    
    Parameters:
        predictions (dict): Dictionary with keys 'T1' and 'T2' mapping to lists of tuples
                            (depth, probability) for predicted fractures.
        output_csv_path (str): Path to save the submission CSV file.
        num_predictions_per_well (int): Maximum number of predictions to include per well.
    """
    submission_data = []
    for well in ['T1', 'T2']:
        # Sort predictions for each well by confidence in descending order
        sorted_preds = sorted(predictions[well], key=lambda x: x[1], reverse=True)
        top_preds = sorted_preds[:num_predictions_per_well]
        for depth, prob in top_preds:
            submission_data.append({'Well': well, 'Depth': depth})
    submission_df = pd.DataFrame(submission_data)
    submission_df.to_csv(output_csv_path, index=False)
    print(f"Submission CSV created at: {output_csv_path}")

# -------------------- Example Usage --------------------

if __name__ == '__main__':
    # Set device and load transformation function
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    transform = get_transforms(train=False)
    
    # Path to the saved ensemble checkpoint file
    ensemble_pth_path = './Machine-Learning-Competition-2024/fracture_identification/ensemble_model.pth'
    
    # Assume that well_images and well_logs dictionaries are already loaded with test well data.
    # For example:
    # well_images = {
    #     'well_T1': pd.read_csv('path/to/T1_image.csv', index_col=0),
    #     'well_T2': pd.read_csv('path/to/T2_image.csv', index_col=0)
    # }
    # well_logs = {
    #     'well_T1': pd.read_csv('path/to/T1_logs.csv', index_col=0),
    #     'well_T2': pd.read_csv('path/to/T2_logs.csv', index_col=0)
    # }
    
    # Generate test views using the sliding window approach
    test_views = generate_test_views(well_images, well_logs, well_list=['T1', 'T2'])
    
    # Run inference on the test wells
    predictions = infer_test_wells(ensemble_pth_path, test_views, transform, device, conf_threshold=0.5)
    
    # Optionally, print predictions
    for well in predictions:
        print(f"\nPredictions for well {well}:")
        for depth, prob in predictions[well]:
            print(f"  Depth: {depth:.2f} m, Confidence: {prob:.4f}")
    
    # Create the submission CSV file
    output_csv_path = './submission.csv'
    create_submission_csv(predictions, output_csv_path, num_predictions_per_well=15)
